# Shortest path
In the last exercise, we showed how to retrieve the supply side inputs of a traffic model: the network. Apart from the supply inputs, traffic models also require the inputs about the travel demand, which is usually specified as origin-destination (OD) pairs. For example, given the OD list below, we know that the first trip starts from node 1 in the road network graph and ends at node 10, so on and so forth. The travel demand inputs are usually obtained based on land use information (e.g., numbers of residential and work units in each census tract). It is a sligthly complicated process that usually differs from projects to projects, so we will not introduce how we generate such data here. Instead, you will be provided with a ready-to-use demand file for your quiz and homework.

| Trip_ID | start_node | end_node |
|---------|------------|----------|
| 1       | 1          | 10       |
| 2       | 15         | 55       |
| ..      | ..         | ..       |

With the supply and demand inputs, traffic simulations can be performed by associating a trip (demands) to a set of road network links that it will use (a path) with a step called `traffic assignment`. There are multiple types of traffic assignments as discussed in the lecture, and you will be practicing coding the simplist one, the `all-or-nothing` assignment in the quiz today. In this exercise, we will prepare you for the quiz by showing how to compute for the shortest path given a trip's origin and destination.

## The `sp` module
There are numerous python packages that can perform the shortest-path calculation, with the most notable ones being [NetworkX](https://networkx.github.io/) and [python-igraph](https://igraph.org/python/). There are multiple [shortest-path finding algorithms](https://en.wikipedia.org/wiki/Shortest_path_problem), while the [Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) is the most generally applicable one. In this class, we will use a specially developed shortest path code, [sp](https://github.com/cb-cities/sp), which implements priority-queue based Dijkstra's Algorithm and has been tested to run more efficiently than other packages.

** the [sp](https://github.com/cb-cities/sp) code is developed by Dr Krishna Kumar at UT Austin (formerly at the Soga group). It computes shortest path using Dijkstra's Algorithm efficiently. If your problem has special features, sometimes using other algorithm can give you even faster results.

** the [sp](https://github.com/cb-cities/sp) code is written in C++ with a Python wrapper. Normally you would need to compile the code on your computer. Here on colab, we provide a compiled dynamic library `liblsp.so`.

In [ ]:
# install some python packages
!pip install geopandas shapely folium

# retrieve the sp code
!rm -rf sp && mkdir sp
!wget "https://github.com/UCB-CE170a/Fall2020/raw/master/traffic_data/liblsp.so" -O sp/liblsp.so
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/interface.py" -O sp/interface.py

# retrieve the road network
!rm -rf traffic_inputs && mkdir traffic_inputs
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/north_berkeley_edges_clean.csv" -O traffic_inputs/berkeley_edges.csv

In [ ]:
from sp import interface
import pandas as pd

In [ ]:
# read edges file
edges_df = pd.read_csv('traffic_inputs/berkeley_edges.csv')
display( edges_df.head(2) )

In [ ]:
# create a graph
# supply the name of the edges dataframe, column name of the start node ID, end node ID and graph weights (free-flow travel time) column
g = interface.from_dataframe(edges_df, 'start_node_id', 'end_node_id', 'fft')

Let's get the path from the CEE department (North gate: osmid 53055202) to Cheeseboard Pizza (Shattuck Avenue and Vine Street: osmid 239617031).

In [ ]:
print( 'The node id of the start location is: ', edges_df.loc[edges_df['start_osmid']==53055202, 'start_node_id'].unique()[0] )
print( 'The node id of the end location is: ', edges_df.loc[edges_df['end_osmid']=='239617031', 'end_node_id'].unique()[0] )

In [ ]:
# get path
def get_path(origin, destin):
    sp = g.dijkstra(origin, destin)
    sp_dist = sp.distance(destin)

    if sp_dist > 10e7:
        route = []
    else:
        route = [(start_sp, end_sp) for (start_sp, end_sp) in sp.route(destin)]
    sp.clear()
    
    return route, sp_dist

origin = 354 ### the origin node id of a trip
destin = 196 ### the end node id of a trip
route, distance = get_path(origin, destin) ### hint: use the provided function `get_path`.
print('The trip travel time is {:.2f} minutes.'.format(distance/60))

In [ ]:
# visualize
import geopandas as gpd
from shapely.geometry import Point
import shapely.wkt
import folium

one_path = pd.DataFrame(route, columns=['start_node_id', 'end_node_id']).merge(
    edges_df[['start_node_id', 'end_node_id', 'geometry']])
one_path_gdf = gpd.GeoDataFrame(one_path, crs='epsg:4326', geometry=one_path['geometry'].map(shapely.wkt.loads))
one_path_json = one_path_gdf.to_json()

start_json = one_path_gdf.iloc[0]['geometry'].coords[0]
end_json = one_path_gdf.iloc[-1]['geometry'].coords[0]

berkeley_map = folium.Map([37.88, -122.25], zoom_start=14)
berkeley_map.add_child(folium.features.GeoJson(one_path_json))
folium.Marker(list(start_json)[::-1], icon = folium.Icon(color='blue')).add_to(berkeley_map)
folium.Marker(list(end_json)[::-1], icon = folium.Icon(color='red')).add_to(berkeley_map)
berkeley_map